# Pandas

In [66]:
import pandas as pd
import yfinance as yf
import numpy as np
from icecream import ic


Group by ~> Separar, aplicar, combinar

<center>
<img src ='groupby.jpg' width="300"></img>
</center>

In [67]:
#groupby para calcular retorno acum
#groupby para calcular o retorno medio por acao
#groupby que calcula o acumulado de lucro do ano por empresa

In [68]:
dados_fundamentalistas = pd.DataFrame({
    'empresa': ['WEGE', 'WEGE', 'WEGE', 'WEGE', 'PETROBRAS', 'PETROBRAS', 'PETROBRAS', 'PETROBRAS'],
    'trimester': ['1TRI', '2TRI', '3TRI', '4TRI'] *2,
    'lucro': [20, 30, 40, 55, 90, 95, 105, 123]})
dados_fundamentalistas

# como calcular o lucro acum do ano?

,empresa,trimester,lucro
0,WEGE,1TRI,20
1,WEGE,2TRI,30
2,WEGE,3TRI,40
3,WEGE,4TRI,55
4,PETROBRAS,1TRI,90
5,PETROBRAS,2TRI,95
6,PETROBRAS,3TRI,105
7,PETROBRAS,4TRI,123


In [69]:
ic(dados_fundamentalistas.groupby('empresa'))
dados_fundamentalistas.groupby('empresa')['lucro'].sum().to_frame()

ic| dados_fundamentalistas.groupby('empresa'): <pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000019674A4CBD0>


,lucro
empresa,
PETROBRAS,413
WEGE,145


In [70]:
acoes = ['WEGE3.SA', 'VALE3.SA', 'PETR4.SA', 'LREN3.SA']
dados_acao = yf.download(acoes, '2019-01-01')['Adj Close']

[*********************100%%**********************]  4 of 4 completed


In [71]:
retornos = dados_acao.pct_change().dropna()
retornos.tail(2)

,LREN3.SA,PETR4.SA,VALE3.SA,WEGE3.SA
Date,,,,
2023-10-13,-0.031298,0.033030,-0.011431,-0.017913
2023-10-16,0.030733,0.002756,0.015768,0.011291


In [72]:
retornos['rent_modelo'] = retornos['WEGE3.SA'] * .25 + retornos['PETR4.SA'] * .25 + retornos['VALE3.SA'] * .25 + retornos['LREN3.SA'] * .25 
retornos #atualmente está no retorno diario do modelo
# como calcular o retorno anual do modelo

,LREN3.SA,PETR4.SA,VALE3.SA,WEGE3.SA,rent_modelo
Date,,,,,
2019-01-03,-0.012629,0.024522,-0.040908,0.022803,-0.001553
2019-01-04,-0.038372,0.002840,0.065102,0.011419,0.010247
2019-01-07,-0.022975,0.015777,-0.005365,-0.015591,-0.007039
2019-01-08,0.032178,-0.005974,0.009632,-0.008192,0.006911
2019-01-09,0.021583,0.020833,0.024423,0.011013,0.019463
...,...,...,...,...,...
2023-10-09,0.007149,0.042972,-0.007179,0.001437,0.011095
2023-10-10,0.025237,0.007439,0.006026,0.012913,0.012904
2023-10-11,0.007692,-0.002556,0.008685,-0.003683,0.002535


In [73]:
retornos['ano'] = retornos.index.year
#criando uma coluna referente ao ano, para agrupar com o groupby
retornos.tail(4)

,LREN3.SA,PETR4.SA,VALE3.SA,WEGE3.SA,rent_modelo,ano
Date,,,,,,
2023-10-10,0.025237,0.007439,0.006026,0.012913,0.012904,2023
2023-10-11,0.007692,-0.002556,0.008685,-0.003683,0.002535,2023
2023-10-13,-0.031298,0.033030,-0.011431,-0.017913,-0.006903,2023
2023-10-16,0.030733,0.002756,0.015768,0.011291,0.015137,2023


In [74]:
#adicionando 1 aos valores do rent diario do modelo
retornos['rent_modelo'] = retornos['rent_modelo'] + 1

#criando uma coluna que acumula os valores de cada ano
retornos['rent_YTD'] = retornos.groupby('ano')['rent_modelo'].cumprod() - 1

retornos

,LREN3.SA,PETR4.SA,VALE3.SA,WEGE3.SA,rent_modelo,ano,rent_YTD
Date,,,,,,,
2019-01-03,-0.012629,0.024522,-0.040908,0.022803,0.998447,2019,-0.001553
2019-01-04,-0.038372,0.002840,0.065102,0.011419,1.010247,2019,0.008678
2019-01-07,-0.022975,0.015777,-0.005365,-0.015591,0.992961,2019,0.001579
2019-01-08,0.032178,-0.005974,0.009632,-0.008192,1.006911,2019,0.008501
2019-01-09,0.021583,0.020833,0.024423,0.011013,1.019463,2019,0.028129
...,...,...,...,...,...,...,...
2023-10-09,0.007149,0.042972,-0.007179,0.001437,1.011095,2023,-0.021781
2023-10-10,0.025237,0.007439,0.006026,0.012913,1.012904,2023,-0.009158
2023-10-11,0.007692,-0.002556,0.008685,-0.003683,1.002535,2023,-0.006647


In [75]:
retornos.groupby('ano').tail(1)

,LREN3.SA,PETR4.SA,VALE3.SA,WEGE3.SA,rent_modelo,ano,rent_YTD
Date,,,,,,,
2019-12-30,0.001604,-0.011140,-0.005597,-0.019242,0.991406,2019,0.446467
2020-12-30,-0.013370,0.002476,0.004364,0.007851,1.000330,2020,0.379288
2021-12-30,0.022594,-0.003153,0.009191,0.007330,1.008991,2021,-0.059653
2022-12-29,-0.017275,-0.012097,-0.001236,-0.004910,0.991121,2022,0.231406
2023-10-16,0.030733,0.002756,0.015768,0.011291,1.015137,2023,0.001429


In [76]:
retornos = dados_acao.pct_change().dropna()
retornos['rent_modelo'] = retornos['WEGE3.SA'] * .25 + retornos['PETR4.SA'] * .25 + retornos['VALE3.SA'] * .25 + retornos['LREN3.SA'] * .25 

retornos['ano'] = retornos.index.year
retornos['mes'] = retornos.index.month

retornos['rent_modelo'] = retornos['rent_modelo'] + 1

retornos['rent_MTD'] = retornos.groupby(['ano', 'mes'])['rent_modelo'].cumprod() - 1

retornos.groupby(['ano', 'mes']).tail(1)[['ano', 'mes', 'rent_MTD']]

,ano,mes,rent_MTD
Date,,,
2019-01-31,2019,1,0.019088
2019-02-28,2019,2,0.007331
2019-03-29,2019,3,0.028751
2019-04-30,2019,4,0.017823
2019-05-31,2019,5,-0.003301
2019-06-28,2019,6,0.081781
2019-07-31,2019,7,0.009617
2019-08-30,2019,8,-0.022892
2019-09-30,2019,9,0.053007


In [77]:
#retorno diario medio por ano
retornos.groupby('ano')['rent_modelo'].mean() -1

ano
2019    0.001580
2020    0.001797
2021   -0.000130
2022    0.000961
2023    0.000101
Name: rent_modelo, dtype: float64

## Desafios

In [91]:
#86
acao = 'MULT3.SA'
dados_acao = yf.download(acao)[['Close', 'Volume']]
dados_acao

[*********************100%%**********************]  1 of 1 completed


,Close,Volume
Date,,
2007-07-27,7.876217,12103289
2007-07-30,7.966212,1887407
2007-07-31,8.036208,6567074
2007-08-01,8.159534,2622449
2007-08-02,8.132869,1934210
...,...,...
2023-10-09,24.950001,4227100
2023-10-10,25.440001,4742800
2023-10-11,25.530001,4758700


In [94]:
dados_acao['Volume em reais'] = dados_acao['Close'] * dados_acao['Volume']
dados_acao['Volume em reais'] = dados_acao['Volume em reais'].astype(int)

dados_acao['ano'] = dados_acao.index.year
dados_acao['mes'] = dados_acao.index.month
dados_acao

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15068\2305261259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_acao['Volume em reais'] = dados_acao['Close'] * dados_acao['Volume']
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15068\2305261259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_acao['Volume em reais'] = dados_acao['Volume em reais'].astype(int)
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15068\2305261259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,Close,Volume,Volume em reais,ano,mes
Date,,,,,
2007-07-27,7.876217,12103289,95328129,2007,7
2007-07-30,7.966212,1887407,15035483,2007,7
2007-07-31,8.036208,6567074,52774373,2007,7
2007-08-01,8.159534,2622449,21397962,2007,8
2007-08-02,8.132869,1934210,15730676,2007,8
...,...,...,...,...,...
2023-10-09,24.950001,4227100,105466148,2023,10
2023-10-10,25.440001,4742800,120656834,2023,10
2023-10-11,25.530001,4758700,121489614,2023,10


In [98]:
volume_medio_mensal = dados_acao.groupby(['ano', 'mes'])['Volume em reais'].mean()
volume_medio_mensal.astype(int)

ano   mes
2007  7       54379328
      8       14133881
      9        3685435
      10       3780835
      11       4772635
               ...    
2023  6      167112233
      7      114731765
      8      105544782
      9      104950916
      10      92502644
Name: Volume em reais, Length: 196, dtype: int32

In [99]:
volume_total_anual = dados_acao.groupby(['ano'])['Volume em reais'].sum()

volume_total_anual

ano
2007      761226932
2008      635478471
2009     1614429694
2010     2647438591
2011     2090236059
2012     4171979775
2013     6543089286
2014     3412838563
2015     9586179887
2016     9463334974
2017    12814125293
2018    13614279714
2019    19851535920
2020    35446265443
2021    37888882326
2022    31145553004
2023    23048823368
Name: Volume em reais, dtype: int64